In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from kfsims.network import create_network, update_node_by_neighbors
from kfsims.common import init_trajectory
import numpy as np

In [32]:
traj = init_trajectory()

In [33]:
NODES = 10

# No fusion

In [45]:
# no fusion
def no_fusion_step_by_step():
    net = create_network(NODES)
    msrms = {node: (i for i in node.observe()) for node in net}
    rmses = []
    for i in range(traj.X.shape[1]):
        for node, ms in msrms.items():
            m = next(ms)
            node.single_kf(m)
    for node in net:
        rmses.append(node.post_rmse(traj.X))
    return np.mean(rmses), net

In [46]:
np.mean([no_fusion_step_by_step()[0] for i in range(10)])

3.7192932396751965

In [47]:
def no_fusion_all_at_once():
    # all at once, should be approx. same as the above
    net = create_network(NODES)
    rmses = []
    for node in net.nodes:
        node()
        rmses.append(node.post_rmse(traj.X))
    return np.mean(rmses)

In [49]:
np.mean([no_fusion_all_at_once() for i in range(10)])

3.7192932396751965

# Simple average

In [50]:
def simple_average():
    net = create_network(NODES)
    msrms = {node: (i for i in node.observe()) for node in net}
    rmses = []
    rm_ = []
    for i in range(traj.X.shape[1]):
        xs = []
        for node, ms in msrms.items():
            m = next(ms)
            x, *_ = node.single_kf(m)
            xs.append(x)
        rm_.append(np.mean(xs, axis=0))
        for node in net:
            update_node_by_neighbors(net, node)        

    for node in net:
        rmses.append(node.post_rmse(traj.X))
    
    #x_log = np.array(rm_).squeeze().T
    #rmse = np.sqrt(((x_log[:2] - traj.X[:2]) ** 2).mean())
    #rmse
    
    return np.mean(rmses)

In [51]:
%%time 
r = [simple_average() for i in range(10)]
print(np.mean(r), np.std(r))

2.6990873427513122 0.0605431439676356
CPU times: user 5.27 s, sys: 11.9 ms, total: 5.28 s
Wall time: 5.3 s


# Simple diffusion 

In [52]:
def simple_diffusion():
    net = create_network(NODES)
    msrms = {node: (i for i in node.observe()) for node in net}
    rmses = []
    for i in range(traj.X.shape[1]):

        # proveď jeden KF loop pro každý node
        for node, ms in msrms.items():
            m = next(ms)
            node.single_kf(m)

        # každému nodu přiřaď průměr hyperparametrů od všech sousedů
        for node in net:
            update_node_by_neighbors(net, node)


    for node in net:
        rmses.append(node.post_rmse(traj.X))
    return np.mean(rmses)

In [53]:
%%time 
r = [simple_diffusion() for i in range(10)]
print(np.mean(r), np.std(r))

2.6764337382942847 0.03943806282647636
CPU times: user 5.3 s, sys: 16.9 ms, total: 5.31 s
Wall time: 5.33 s
